# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: The type of this supervised learning problem is Classification, because the target output is descrete once it will be identified which of the students will need an early intervention before they fail to graduate (class 1) and which won't need it (class 2). Therefore, it will be trained a model capable of classifying a student between these two classes (need early intervention (failed) and don't need an early intervention(passed)) based on their academic performance. **

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
#n_students = None
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = (student_data.shape[1]-1)

# TODO: Calculate passing students
n_passed = len(student_data[(student_data['passed'] == 'yes')])

# TODO: Calculate failing students
n_failed = len(student_data[(student_data['passed'] == 'no')])

# TODO: Calculate graduation rate
grad_rate = (float(n_passed)/n_students)*(100)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split
# TODO: Set the number of training points
num_train = 300
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test,stratify=y_all, random_state=42)
#http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html suggestion

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer:**
The three supervised learning models selected were: Gaussian Naive Bayes (GaussianNB), Random Forest and Logistic Regression. 

**1-Gaussian Naive Bayes (GaussianNB)**
One real world application of Gaussian Naive Bayes (GaussianNB) is the classification of an email as spam/ not spam. The Naive Bayes assumption is that all the features are conditionally independent given the class label, even though this is usually false. This assumption allows the calculation of the probability of being spam given that we know the features (main words, spelling mistakes,...) within the email. The Bayesian classifier predicts that the e-mail belongs to class spam if the probability of the email have the determined features(certain words such as viagra, deal, and alsospelling mistakes) given that it is a spam email (posterior probability)be bigger than the probability of the email have the same determined features given that it is a no spam email.

    Strengths of Gaussian Naive Bayes (GaussianNB):  It is easy to implemente, requires a small amount of training data to estimate the parameters, good results obtained in most of the cases, good computational efficiency and good performance for real-world problems. Studies comparing classification algorithms have found that the naive Bayesian classifier to be comparable in performance with decision tree and selected neural network classifiers. Bayesian classifiers have also exhibited high accuracy and speed when applied to large databases.
    Weaknesses of Gaussian Naive Bayes (GaussianNB): the assumption of the class conditional independence, therefore loss of accuracy. Practically, dependencies exist among variables (for example, in hospitals the patients have in their profile age, family historic, symptoms, disease, and these have dependencies) and dependencies among these cannot be modelled by Naive Bayesian Classifier. So when features are not independent given the class label, naive Bayes will make wrong decisions. If the dependency between the features is too Strong, naive Bayes might perform poorly.

I belive that the naive bayes will perform reasonably once neither of the classes have a to small amoount of samples, even though some of the features might have dependencies. Also, the amound of the dataset seems to be sufficient to train the model. 
**References:**
https://pt.slideshare.net/ashrafmath/naive-bayes-15644818
https://stats.stackexchange.com/questions/77645/in-what-conditions-does-naive-bayes-classifier-perform-poorly

**2- Decision Trees**
One real world application of decision trees classifier is the algorithm created to identify cosmic-ray hits in Hubble Space Telescope images. In this case, decision trees classifiers was producing results with over 95% accuracy using data from a single, unpaired image. The main goal of this classifier is to eliminate the noise that cosmic rays cause in the images taken by the telescope, which hinders the identification of stars.

    Advantages of Decision Trees: This model can generate understandable rules, perform classification without much computation,	can also handle continuous and categorical variables and, finally,provide a clear indication of which fields are most important for prediction or classification. 
    Weaknesses of Decision Trees: Decision trees perform very poorly with datasets that have many classes and small data. It is not suitable for prediction of continuous attribute and is computationally expensive to train.It also do not treat well non-rectangular regions, for data including categorical variables with different number of levels, information gain in decision trees is biased in favor of those attributes with more levels ans calculations can get very complex, particularly if many values are uncertain and/or if many outcomes are linked.

For this classification problem,the size of the dataset seems to be sufficient to make this model perform weel. In addition,most of the dataset variables look adequated and do not seem to have a dependency between eachother. Therefore, I consider that this model will perform nicely and give a good result.
**References:** 
http://adsbit.harvard.edu//full/1995PASP..107..279S/0000279.000.html
https://en.wikipedia.org/wiki/Decision_tree#Advantages_and_disadvantages

**3- Random Forest**
This model can be applied into tumor classification by analyzing many features such as age, weight, sex and so on, as well as tissue microarray (protein abundancies) profiling, which would allow the identification between classes such as benign and malignant, or several other types of clusters. These analyses are important in order to avoid unnecessary chemotherapy or the cancer progression. Also, by analyzing diagnosed image biopsy even more features can be extract and used in the analysis, enriching the data and improving the model.

    Strengths of Random Forest: As strengths, we have that this model is one of the most accurate learning algorithms available.  For many datasets, it produces highly accurate classifier. It performs efficiently on large databases and it can handle thousands of input variables without variable deletion. It performs pretty well on data where there is no strong relationship between the variables. It also gives estimates of what variables are important in the classification.
    Weaknesses of Random Forest: The weaknesses of this model are that they have been observed to overfit for some datasets with noisy classification/ regression tasks. Also for data including categorical variables with different number of levels, random forests are biased in favor of those attributes with more levels. Therefore, the variable importance scores from random forest are not reliable for this type of data.  If the dataset have few examples and many variables, it will also not perform nicely.**
    
This model is a good candidate for this problem, once we have a good size of data set with many features, the dataset also does not seem to be noisy, most of the categorical variables have only two levels (binary) and the relationship among the variables does not seem to be strong. 
**References:** 
https://www.nature.com/articles/3800322
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5575012/
https://ti.arc.nasa.gov/m/profile/oza/files/oztu08.pdf
https://stats.stackexchange.com/questions/112148/when-to-avoid-random-forest
http://slideplayer.com/slide/3274501/

https://static.coggle.it/diagram/WHeBqDIrJRk-kDDY/t/categories-of-algorithms-non-exhaustive

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from sklearn import model_C
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# TODO: Initialize the three models
#clf_A = GaussianNB(priors=[0.1, 0.9])#priors=[0.1, 0.9]
clf_A = GaussianNB()#priors=[0.1, 0.9]
#clf_B = DecisionTreeClassifier(min_samples_split=3,max_depth = 2,random_state=0)
clf_B = DecisionTreeClassifier(random_state=0)
#clf_C = RandomForestClassifier(max_depth=2, random_state=0)
clf_C = RandomForestClassifier(random_state=0)

# TODO: Set up the training set sizes

#X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(X_train, y_train,train_size=100, test_size= None, random_state=42)
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]
#X_train_200, X_test_200, y_train_200, y_test_200 = train_test_split(X_train, y_train,train_size=200, test_size= None, random_state=42)
X_train_200 = X_train[:200]
y_train_200 = y_train[:200]
#X_train_300, X_test_300, y_train_300, y_test_300 = train_test_split(X_train, y_train,train_size=300, test_size= None, random_state=42)
X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.7752.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.6457.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0020 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8060.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7218.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0010 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8134.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7761.
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6667.
Training a DecisionTreeClassifier using a training set size of 200. 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - ?**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0010          |        0.0000          |   0.7752         |     0.6457      |
| 200               |         0.0020          |        0.0020          |   0.8060         |     0.7218      |
| 300               |         0.0010          |        0.0010          |   0.8134         |     0.7761      |

** Classifer 2 - ?**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0010         |         0.0010         |      1.0000      |     0.6667      |
| 200               |          0.0020         |         0.0010         |      1.0000      |     0.7097      |
| 300               |          0.0020         |         0.0000         |      1.0000      |     0.6557      |

** Classifer 3 - ?**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0220          |         0.0010         |     0.9849       |    0.6984       |
| 200               |         0.0230          |         0.0030         |     0.9845       |    0.7368       |
| 300               |         0.0220          |         0.0030         |     0.9828       |    0.7612       |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: The model that I chose as the best one is the Decision Tree. By analyzing the results given by all the models, without changing any parameter, the Decision tree has shown a good training time and a good prediction (test) time, which both are very close to the times measured in the GaussianNB, and way smaller than the RandomForestClassifier. Another nice characteristic to pinpoint is that, once the model has overfitted, presenting a F1 Score equals 1, it is possible do conclude that the model's performance can be  considerably improved by exploring its parameters and making it even better than the results of the other two models. Also, the GaussianNB does not have parameters that make its improvement results possible and it is less eaysier to interpret, which also make the Decision Tree more interesting to explore. Finally, comparing the cost of the Decision Tree and the Random Forest models, the Decision tree has a smallest processing cost, once is an easier and faster model to implement and train. 
It is important to say that I have tested some parameters and 2 values of each, which helped me be more certain about the possible improvement of this model.**

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer:** One of the best thigs about decision trees is that they are easy to comprehand and interpret. The algorithm of a decision tree is a very simple thing and consist on the next steps:

First, we place the best attribute (the one that best devides the dataset, where the best division would be into two roughly equal chunks) at the root of the tree. Than, we ask a question about it and, depending upon the answer, we went and pick another attribute, which should be the best next attribute, and follow the answer path. We do this until we narrow the space of possibilities to one item, and by doing this continously we form branches that leads to different leaves, which represent the final answer of our questions. This first process consists on the learning process of our decision tree and, as we can see, this model works in a very transparent way, making it even better to understand in a deeper sense. 

After the learning process, we have the classification/prediction process. With our trained decision tree, given one new situation where we have to predict which is the most probable answer, we analyse each  node regarding this new situation, and follow the path given the answers of these nodes until we reach one of the leaves and find the prediction result. 


### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [19]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier

# TODO: Create the parameters list you wish to tune
#criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, 
#min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, 
#max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False
#parameters = [{'min_samples_split': [2,3, 4, 5],'max_depth': [2,3, 4, 5],
#    'min_samples_leaf': [1, 2, 3, 4], 'random_state': [0]
# }]
parameters = [{'max_features': [5,10, 15, 20,25,30],'min_samples_split': [2,3, 4, 5,6,7],'max_depth': [3,2, 4, 5,6,7],
     'random_state': [0]
      }]         
# TODO: Initialize the classifier
clf = DecisionTreeClassifier()
#DecisionTreeClassifier(min_samples_split=3,max_depth = 2,random_state=0)
# TODO: Make an f1 scoring function using 'make_scorer' 
# https://stackoverflow.com/questions/34221712/grid-search-with-f1-as-scoring-function-several-pages-of-error-message
f1_scorer = make_scorer(f1_score, pos_label="yes")
#reg = grid_search.GridSearchCV(estimator=neigh,param_grid=parameters,scoring=f1_scorer)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,param_grid=parameters,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0010 seconds.
Tuned model has a training F1 score of 0.8477.
Made predictions in 0.0000 seconds.
Tuned model has a testing F1 score of 0.7972.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: The final model's F1 Score for training and testing are, respectively, 0.8477 and 0.7972, which is a way better value than the result that we got with the untuned model, which was 1.000 and 0.6557 for the training and testing sets, respectively. The untuned model result clearly showed a overfitted model, indicating a high variance. It is also possible to see an improvement in the training time, even though the result previously shown was already good. 
Concluding, by seeing the results obtained throughout this project, it was possible to confirm and see in practice the importance of the analysis around the untuned model to diagnose the possible problems and try different approaches to make it better.**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.